# Ataca la API de Marvel

![imagen](https://cdn.hobbyconsolas.com/sites/navi.axelspringer.es/public/styles/hc_1440x810/public/media/image/2021/09/marvel-2460339.jpg?itok=cBnC1CBi)

Te tendrás que [registrar](https://developer.marvel.com/) y consultar la [documentación](https://developer.marvel.com/docs)

Queremos que consultes a la api para que te devuelva la info de los personajes de marvel que empiecen por la inicial de tu nombre. Tendríamos que generar un main.py (más los scripts que se te ocurran) que guarde un csv la información con la siguiente estructura

![imagen](./img/Captura_marvel.PNG)

Aquí te dejamos el código en python para poder empezar a hacer las llamadas, rellenando algunas variables como tus keys, parámetros de la llamada y la url (endpoint) a la que quieres acceder

In [ ]:
import requests
import hashlib
import time
import csv
import os
import hashlib
import requests
import datetime
import pandas as pd

In [ ]:
pd.set_option("max_colwidth", 100)

In [ ]:
PUBLIC_KEY = ""
PRIVATE_KEY = ""
INITIAL = "A" 
OUTPUT_FILE = "marvel_characters.csv"
LIMIT = 100  

In [ ]:
#CONECTAR API
class MarvelAPI:
    def __init__(self, public_key: str, private_key: str):
        self.public_key = public_key
        self.private_key = private_key
        self.base_url = "https://gateway.marvel.com/v1/public/characters"

    def _get_auth_params(self):
        ts = str(time.time())
        hash_md5 = hashlib.md5(f"{ts}{self.private_key}{self.public_key}".encode()).hexdigest()
        return {"ts": ts, "apikey": self.public_key, "hash": hash_md5}

    def get_characters_by_initial(self, initial: str):
        all_characters = []
        offset = 0

        while True:
            params = self._get_auth_params()
            params.update({"nameStartsWith": initial, "limit": LIMIT, "offset": offset})
            response = requests.get(self.base_url, params=params)
            response.raise_for_status()
            data = response.json()
            results = data.get("data", {}).get("results", [])
            all_characters.extend(results)
            
            total = data.get("data", {}).get("total", 0)
            count = data.get("data", {}).get("count", 0)
            offset += count
            
            if offset >= total or count == 0:
                break

        return all_characters

In [ ]:
# -------------------------------

def main():
    initial = input("Introduce la inicial de los personajes de Marvel que quieres buscar: ").strip().upper()
    
    if not initial.isalpha() or len(initial) != 1:
        print("Por favor, introduce solo una letra válida.")
        return

    marvel = MarvelAPI(PUBLIC_KEY, PRIVATE_KEY)
    print(f"Buscando personajes que comienzan con '{initial}'...")
    try:
        characters = marvel.get_characters_by_initial(initial)
        save_characters_to_csv(characters, OUTPUT_FILE)
        print(f"{len(characters)} personajes guardados en '{OUTPUT_FILE}'")
    except requests.exceptions.HTTPError as e:
        print("Error al consultar la API:", e)
        print("Revisa tus claves de Marvel y tu conexión a internet.")

if __name__ == "__main__":
    main()

Buscando personajes que comienzan con 'A'...
Error al consultar la API: 401 Client Error: Unauthorized for url: https://gateway.marvel.com/v1/public/characters?ts=1761298303.8870466&apikey=TU_PUBLIC_KEY&hash=6a23c542b171b1421f23eb848b6e8320&nameStartsWith=A&limit=100&offset=0
Revisa tus claves de Marvel y tu conexión a internet.


In [ ]:
characters = [
  
    {"id": 1010914, "name": "Captain America (House of M)", "thumbnail": {"path": "http://i.annihil.us/u/prod/marvel/i/mg/6/10/53176a1e7c0d5", "extension": "jpg"}},
    {"id": 1017295, "name": "Captain America (LEGO Marvel Super Heroes)", "thumbnail": {"path": "http://i.annihil.us/u/prod/marvel/i/mg/d/b0/5239c38051946", "extension": "jpg"}},
    {"id": 1017327, "name": "Captain America (Marvel War of Heroes)", "thumbnail": {"path": "http://i.annihil.us/u/prod/marvel/i/mg/2/03/5239c005a4827", "extension": "jpg"}}]


data = []
for char in characters:
    thumbnail = char.get("thumbnail", {})
    path = thumbnail.get("path", "")
    ext = thumbnail.get("extension", "")
    picture_url = f"{path}.{ext}" if path and ext else None
    data.append({
        "id": char.get("id"),
        "name": char.get("name"),
        "picture_url": picture_url})


In [ ]:
df = pd.DataFrame(data)

print(df.head(10))

,id,name,picture_url
0,1010914,Captain America (House of M),http://i.annihil.us/u/prod/marvel/i/mg/6/10/53176a1e7c0d5.jpg
1,1017295,Captain America (LEGO Marvel Super Heroes),http://i.annihil.us/u/prod/marvel/i/mg/d/b0/5239c38051946.jpg
2,1017327,Captain America (Marvel War of Heroes),http://i.annihil.us/u/prod/marvel/i/mg/2/03/5239c005a4827.jpg
3,1017575,Captain America (Sam Wilson),http://i.annihil.us/u/prod/marvel/i/mg/c/80/545a84a75ddaf.jpg
4,1010913,Captain America (Ultimate),http://i.annihil.us/u/prod/marvel/i/mg/5/80/53176a0bb810c.jpg
...,...,...,...
95,1011333,Dakota North,http://i.annihil.us/u/prod/marvel/i/mg/7/d0/4ce5a6888f769.jpg
96,1009259,Damage Control,http://i.annihil.us/u/prod/marvel/i/mg/5/00/4c00404b5e1a2.jpg
97,1009453,Dani Moonstar,http://i.annihil.us/u/prod/marvel/i/mg/6/20/4ce5a0614487a.jpg
98,1010776,Danny Rand,http://i.annihil.us/u/prod/marvel/i/mg/2/20/4c00374b1008a.jpg


In [ ]:
def get_all_characters(initial: str):
    base_url = "https://gateway.marvel.com/v1/public/characters"
    all_characters = []
    offset = 0

    while True:
        ts = str(time.time())
        hash_md5 = hashlib.md5(f"{ts}{PRIVATE_KEY}{PUBLIC_KEY}".encode()).hexdigest()
        params = {
            "apikey": PUBLIC_KEY,
            "ts": ts,
            "hash": hash_md5,
            "nameStartsWith": initial,
            "limit": LIMIT,
            "offset": offset}

        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()["data"]
        results = data["results"]

        # Extraer id, name y picture_url
        for char in results:
            thumbnail = char.get("thumbnail", {})
            path = thumbnail.get("path", "")
            ext = thumbnail.get("extension", "")
            picture_url = f"{path}.{ext}" if path and ext else None
            all_characters.append({
                "id": char.get("id"),
                "name": char.get("name"),
                "picture_url": picture_url})

        offset += data["count"]
        if offset >= data["total"]:
            break

    return all_characters


reset_index(...): limpia y reordena el índice del DataFrame.

if __name__ == "__main__": main(): indica “si este archivo es el programa principal, arranca la función main()”. Si no, solo actúa como librería.

In [ ]:
def main():
    initial = input("Introduce la inicial de los personajes: ").strip().upper()
    
    if not initial.isalpha() or len(initial) != 1:
        print("Por favor, introduce solo una letra válida.")
        return

    print(f"Descargando todos los personajes que comienzan con '{initial}'...")
    characters = get_all_characters(initial)


    df = pd.DataFrame(characters)


    df.sort_values(by=["id", "name"], inplace=True)

    # Resetear índices
    df.reset_index(drop=True, inplace=True)


if __name__ == "__main__":
    main()


,id,name,picture_url
0,1011334.0,3-D Man,http://i.annihil.us/u/prod/marvel/i/mg/c/e0/535fecbbb9784.jpg
1,1017100.0,A-Bomb (HAS),http://i.annihil.us/u/prod/marvel/i/mg/3/20/5232158de5b16.jpg
2,1009144.0,A.I.M.,http://i.annihil.us/u/prod/marvel/i/mg/6/20/52602f21f29ec.jpg
3,1010699.0,Aaron Stack,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
4,1009146.0,Abomination (Emil Blonsky),http://i.annihil.us/u/prod/marvel/i/mg/9/50/4ce18691cbf04.jpg
...,...,...,...
1295,1009631.0,Sue Storm,http://i.annihil.us/u/prod/marvel/i/mg/6/a0/52695b9cd40b6.jpg
1296,1011212.0,Sugar Man,http://i.annihil.us/u/prod/marvel/i/mg/f/d0/527413a2480b5.jpg
1297,1009635.0,Sumo,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
1298,1009636.0,Sunfire,http://i.annihil.us/u/prod/marvel/i/mg/7/00/53629b9e24ef5.jpg


In [ ]:
def main():
    initial = input("Introduce la inicial de los personajes: ").strip().upper()
    
    if not initial.isalpha() or len(initial) != 1:
        print("Por favor, introduce solo una letra válida.")
        return

    print(f"Descargando todos los personajes que comienzan con '{initial}'...")
    characters = get_all_characters(initial)

   
    df = pd.DataFrame(characters)

    
    df.sort_values(by=["name", "id"], ascending=[False, True], inplace=True)


    df.reset_index(drop=True, inplace=True)


    print(df.head(10))

if __name__ == "__main__":
    main()


,id,name,picture_url
0,1009637.0,Sunset Bain,http://i.annihil.us/u/prod/marvel/i/mg/c/10/4c003ab9ed7d6.jpg
1,1009638.0,Sunspot,http://i.annihil.us/u/prod/marvel/i/mg/b/50/4c003ab716add.jpg
2,1015280.0,Super Hero Squad,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
3,1011022.0,Super-Adaptoid,http://i.annihil.us/u/prod/marvel/i/mg/5/e0/4c00325af279c.jpg
4,1009639.0,Super-Skrull,http://i.annihil.us/u/prod/marvel/i/mg/9/b0/5274138fe6deb.jpg
...,...,...,...
257,1011515.0,Zeus,http://i.annihil.us/u/prod/marvel/i/mg/f/60/4ce5a7fcaa386.png
258,1011127.0,Zodiak,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
259,1009741.0,Zombie (Simon Garth),http://i.annihil.us/u/prod/marvel/i/mg/6/10/4c003937c9ba4.jpg
260,1011183.0,Zuras,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg


## Encapsúlalo en un main.py, funciones.py y variables.py

In [ ]:
from funciones import get_all_characters, create_dataframe, save_dataframe
from variables import OUTPUT_FILE

import requests
import hashlib
import time
import pandas as pd
from variables import PUBLIC_KEY, PRIVATE_KEY, LIMIT

In [ ]:
## Encapsulado 
marvel_project/
├─ main.py
├─ funciones.py
└─ variables.py

In [ ]:

PUBLIC_KEY = ""
PRIVATE_KEY = ""

LIMIT = 100  
OUTPUT_FILE = "marvel_characters_descending.csv"

In [ ]:


def get_all_characters(initial: str):
    """
    Obtiene todos los personajes de Marvel que comienzan con la inicial dada.
    Devuelve una lista de diccionarios con id, name y picture_url.
    """
    base_url = "https://gateway.marvel.com/v1/public/characters"
    all_characters = []
    offset = 0

    while True:
        ts = str(time.time())
        hash_md5 = hashlib.md5(f"{ts}{PRIVATE_KEY}{PUBLIC_KEY}".encode()).hexdigest()
        params = {
            "apikey": PUBLIC_KEY,
            "ts": ts,
            "hash": hash_md5,
            "nameStartsWith": initial,
            "limit": LIMIT,
            "offset": offset
        }

        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()["data"]
        results = data["results"]

        # Extraer id, name y picture_url
        for char in results:
            thumbnail = char.get("thumbnail", {})
            path = thumbnail.get("path", "")
            ext = thumbnail.get("extension", "")
            picture_url = f"{path}.{ext}" if path and ext else None
            all_characters.append({
                "id": char.get("id"),
                "name": char.get("name"),
                "picture_url": picture_url
            })

        offset += data["count"]
        if offset >= data["total"]:
            break

    return all_characters

def create_dataframe(characters):
    """
    Convierte la lista de personajes en un DataFrame y lo ordena de la S a la Z.
    """
    df = pd.DataFrame(characters)
    
    df.sort_values(by=["name", "id"], ascending=[False, True], inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

def save_dataframe(df, filename):
    """
    Guarda el DataFrame en un CSV.
    """
    df.to_csv(filename, index=False)


In [ ]:


def main():
    initial = input("Introduce la inicial de los personajes de Marvel: ").strip().upper()
    
    if not initial.isalpha() or len(initial) != 1:
        print("Por favor, introduce solo una letra válida.")
        return

    print(f"Descargando personajes que comienzan con '{initial}'...")
    try:
        characters = get_all_characters(initial)
        df = create_dataframe(characters)
        save_dataframe(df, OUTPUT_FILE)
        print(f"{len(df)} personajes guardados en '{OUTPUT_FILE}'")
        print(df.head(10))  # muestra ejemplo de los primeros 10
    except Exception as e:
        print("Error al consultar la API o procesar los datos:", e)

if __name__ == "__main__":
    main()
